In [2]:
import pandas as pd
import os
import random
from concurrent.futures import ThreadPoolExecutor

import cv2 as cv
import numpy as np
import pyvips
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm

In [3]:

projects = ["BRCA", "COAD", "GBM", "HNSC", "LIHC", "LUAD", "LUSC", "OV", "PRAD", "THCA", "UCEC"]

mut_info = pd.read_csv("mut_info.tsv", sep="\t")
mut_info = mut_info.reset_index()
mut_info.columns = ['Patient ID'] + mut_info.columns[1:].tolist()
mut_info["Patient ID"] = mut_info["Patient ID"].str.rsplit('-', n=1).str[0]
pID_mut = list(mut_info["Patient ID"].unique())

summary_data = []

for project in projects:
    filename = f"data/TCGA-{project}_summary.csv"
    if not os.path.exists(filename):
        print(f"File {filename} not found. Skipping.")
        continue

    proj_data = pd.read_csv(filename, skiprows=2)

    proj_data = proj_data.dropna(subset=["Patient ID", "Diagnostic Slides"])

    proj_data = proj_data[proj_data["Diagnostic Slides"] != 0]

    proj_data["Patient ID"] = proj_data["Patient ID"].str.strip()

    pID_proj = list(proj_data["Patient ID"].unique())

    common_patients = sorted(set(pID_mut) & set(pID_proj))
    non_common_patients = sorted(set(pID_proj) - set(pID_mut))

    common_df = pd.DataFrame({"Patient ID": common_patients})
    common_df.to_csv(f"common_patients_{project}.csv", index=False)

    summary_data.append({
        "Project": project,
        "Total Patients in Summary": len(pID_proj),
        "Common with mut_info": len(common_patients),
        "Non-common in Summary": len(non_common_patients)
    })

    print(f"Processed {project}: {len(common_patients)} common, {len(non_common_patients)} non-common")


# Save universal summary CSV
summary_df = pd.DataFrame(summary_data)
summary_df.to_csv("common_patients_summary.csv", index=False)


Processed BRCA: 1011 common, 51 non-common
Processed COAD: 419 common, 32 non-common
Processed GBM: 94 common, 295 non-common
Processed HNSC: 419 common, 31 non-common
Processed LIHC: 349 common, 16 non-common
Processed LUAD: 461 common, 17 non-common
Processed LUSC: 456 common, 22 non-common
Processed OV: 72 common, 34 non-common
Processed PRAD: 382 common, 21 non-common
Processed THCA: 443 common, 63 non-common
Processed UCEC: 482 common, 23 non-common


Data pref for tile generation

In [4]:
mut_info = pd.read_csv("mut_info.tsv", sep="\t")
mut_info = mut_info.reset_index()
mut_info.columns = ['Patient ID'] + mut_info.columns[1:].tolist()
mut_info["Patient ID"] = mut_info["Patient ID"].str.rsplit('-', n=1).str[0]

In [5]:
binary_mute_info = mut_info.loc[:, mut_info.columns != 'Patient ID'].applymap(lambda x: 1 if x != "wild" else 0)
binary_mute_info.insert(0, 'Patient ID', mut_info['Patient ID'])

/tmp/ipykernel_2166101/249296531.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  binary_mute_info = mut_info.loc[:, mut_info.columns != 'Patient ID'].applymap(lambda x: 1 if x != "wild" else 0)


Now we want to exlude patientIDs not in our Ovarian Cacner data.

In [6]:
common_OV_patients = pd.read_csv("common_patients_OV.csv")
binary_mute_info = binary_mute_info[binary_mute_info["Patient ID"].isin(common_OV_patients["Patient ID"])]

Time to generete tiles for each patient CSV. I will use my previous code for LungInsight to geneate tiles.

In [7]:
def pyvips_to_numpy(vips_image):
    return np.ndarray(buffer=vips_image.write_to_memory(),
                      dtype=np.uint8,
                      shape=[vips_image.height, vips_image.width, vips_image.bands])

In [8]:
def process_tile_2(x, y, width, height, patch_size_w, patch_size_h, lower_bnd_intensity, upper_bnd_intensity, file_path, output_dir, slide):
    patch_coordinates = []

    actual_patch_w = min(patch_size_w, width - x)
    actual_patch_h = min(patch_size_h, height - y)

    patch_coordinates.append((x, y, actual_patch_w, actual_patch_h))

    tile = slide.crop(x, y, actual_patch_w, actual_patch_h)
    tile_array = pyvips_to_numpy(tile)
    mean_value = np.mean(tile_array)

    if lower_bnd_intensity < mean_value <= upper_bnd_intensity:
        return None
    
    output_filename = os.path.expanduser(f"{output_dir}/{output_dir.split("/")[-1].split("_")[0]}_tile_{x}_{y}.png")
    tile.pngsave(output_filename, compression=9)

    df = binary_mute_info[binary_mute_info['Patient ID'] == output_dir.split("/")[-1].split("_")[0]]
    if (not os.path.exists(os.path.expanduser(f"{output_dir}/{output_dir.split("/")[-1].split("_")[0]}.csv")) and not df.empty):
        df.to_csv(os.path.expanduser(f"{output_dir}/{output_dir.split("/")[-1].split("_")[0]}.csv"), index=False)

    return output_filename

In [9]:
def generate_tiles(width, height, patch_size_w, patch_size_h, lower_bnd_intensity, upper_bnd_intensity, file_path, output_dir, slide):
    with ThreadPoolExecutor() as executor:
        futures = []
        for y in range(0, height, patch_size_h):
            for x in range(0, width, patch_size_w):
                futures.append(executor.submit(process_tile_2, x, y, width, height,
                                               patch_size_w, patch_size_h,
                                               lower_bnd_intensity, upper_bnd_intensity,
                                               file_path, output_dir, slide))
        for future in futures:
            _ = future.result()

In [10]:
# ==== PARAMETERS ====
patch_size_w = 1024
patch_size_h = 1024
lower_bnd_intensity = 210
upper_bnd_intensity = 255

# ==== PATHS ====
base_slide_path = os.path.expanduser("~/Documents/Data/TCGAOV/tcga_data/slides/TCGA-OV/")
tile_output_base = os.path.expanduser("~/Documents/Data/TCGAOV/tcga_data/tiles/")

# ==== GET VALID PATIENT IDs ====
valid_patient_ids = set(binary_mute_info["Patient ID"])

# ==== WALK THROUGH FOLDERS ====
for patient_folder in tqdm(os.listdir(base_slide_path), desc="Processing Patient Folders"):
    full_patient_path = os.path.join(base_slide_path, patient_folder)

    # Skip if not in common patients
    if patient_folder not in valid_patient_ids:
        continue

    if not os.path.isdir(full_patient_path):
        continue

    svs_files = [f for f in os.listdir(full_patient_path) if f.endswith(".svs")]
    if not svs_files:
        continue

    for svs_file in svs_files:
        full_svs_path = os.path.join(full_patient_path, svs_file)
        slide = pyvips.Image.new_from_file(full_svs_path, level=1)
        width = slide.width
        height = slide.height

        print(f"Processing {svs_file} for patient {patient_folder}: {width}x{height}")

        output_dir = os.path.join(tile_output_base, f"{patient_folder}_{os.path.splitext(svs_file)[0]}")
        os.makedirs(output_dir, exist_ok=True)

        generate_tiles(
            width=width,
            height=height,
            patch_size_w=patch_size_w,
            patch_size_h=patch_size_h,
            lower_bnd_intensity=lower_bnd_intensity,
            upper_bnd_intensity=upper_bnd_intensity,
            file_path=full_svs_path,
            output_dir=output_dir,
            slide=slide
        )

Processing Patient Folders:   0%|          | 0/72 [00:00<?, ?it/s]

Processing TCGA-23-1118-01Z-00-DX1.D69E130E-33C7-4F8A-99AF-400B81378578.svs for patient TCGA-23-1118: 27389x16629


Processing Patient Folders:   1%|▏         | 1/72 [00:24<28:56, 24.46s/it]

Processing TCGA-OY-A56Q-01Z-00-DX1.F1556F26-8845-4962-9004-70F9747B46B7.svs for patient TCGA-OY-A56Q: 38317x11164


Processing Patient Folders:   3%|▎         | 2/72 [00:43<25:04, 21.49s/it]

...

Processing Patient Folders:  97%|█████████▋| 70/72 [43:10<00:46, 23.31s/it]

Processing TCGA-25-1326-01Z-00-DX1.686E3548-8891-42ED-ABBA-BE3DA6D4CAAB.svs for patient TCGA-25-1326: 20926x19568


Processing Patient Folders:  99%|█████████▊| 71/72 [43:25<00:20, 20.73s/it]

Processing TCGA-23-1113-01Z-00-DX1.D72B1587-444A-4DEC-8D92-703699822BE2.svs for patient TCGA-23-1113: 25895x19207


Processing Patient Folders: 100%|██████████| 72/72 [43:44<00:00, 36.45s/it]


'TCGA-23-1113'

# Using `Virchow2` to 1- Transform into new embeddings  2- Cluster tiles to see what will be clusters at the end.
Virchow2 is a self-supervised vision transformer pretrained using 3.1M whole slide histopathology images. The model can be used as a tile-level feature extractor (frozen or finetuned) to achieve state-of-the-art results for a wide variety of downstream computational pathology use cases.

[link](https://huggingface.co/paige-ai/Virchow2)

In [ ]:
from dotenv import load_dotenv
load_dotenv()
TOKEN = os.getenv("HUGGINGFACE_TOKEN")

In [ ]:
import os
import torch
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.layers import SwiGLUPacked
from PIL import Image
import numpy as np
from huggingface_hub import login
from sklearn.cluster import KMeans
import glob
import shutil

login(token= TOKEN)  

model = timm.create_model("hf-hub:paige-ai/Virchow2", pretrained=True, mlp_layer=SwiGLUPacked, act_layer=torch.nn.SiLU)
model = model.eval() 

transforms = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))

def extract_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transforms(image).unsqueeze(0) 
    with torch.inference_mode():
        output = model(image)  
    class_token = output[:, 0]
    patch_tokens = output[:, 5:]
    embedding = torch.cat([class_token, patch_tokens.mean(1)], dim=-1)
    return embedding.cpu().numpy()

tile_dir = "images" 
output_dir_cluster0 = "Class 0"  
output_dir_cluster1 = "Class 1"  
output_dir_cluster2 = "Class 2"  
output_dir_cluster3 = "Class 3"  
os.makedirs(output_dir_cluster0, exist_ok=True)
os.makedirs(output_dir_cluster1, exist_ok=True)
os.makedirs(output_dir_cluster2, exist_ok=True)
os.makedirs(output_dir_cluster3, exist_ok=True)

tile_paths = glob.glob(os.path.join(tile_dir, "*.png"))
embeddings = []
for tile_path in tile_paths:
    embedding = extract_embedding(tile_path)
    embeddings.append(embedding)

embeddings = np.vstack(embeddings)

kmeans = KMeans(n_clusters=4, random_state=42)
cluster_labels = kmeans.fit_predict(embeddings)

for tile_path, label in zip(tile_paths, cluster_labels):
    tile_name = os.path.basename(tile_path)
    if label == 0:
        shutil.copy(tile_path, os.path.join(output_dir_cluster0, tile_name))
    elif label == 1:
        shutil.copy(tile_path, os.path.join(output_dir_cluster1, tile_name))
    elif label == 2:
        shutil.copy(tile_path, os.path.join(output_dir_cluster2, tile_name))
    else:
        shutil.copy(tile_path, os.path.join(output_dir_cluster3, tile_name))

print(f"Tiles clustered into {output_dir_cluster0}, {output_dir_cluster1}, {output_dir_cluster2}, and {output_dir_cluster3}.")

Tiles clustered into Class 0, Class 1, Class 2, and Class 3. Inspect clusters to identify cancerous tiles.
